# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments  \
0  Dec 14 2022  2:28PM  253126        19     60018253   
1  Dec 14 2022  2:24PM  253125        16      8898126   
2  Dec 14 2022  2:22PM  253124        16      8901822   
3  Dec 14 2022  2:19PM  253123        19  ADV80009039   
4  Dec 14 2022  2:00PM  253122        10    MSP216761   
5  Dec 14 2022  2:00PM  253122        10    MSP216762   
6  Dec 14 2022  2:00PM  253122        10    MSP216763   
7  Dec 14 2022  2:00PM  253122        10    MSP216764   
8  Dec 14 2022  2:00PM  253122        10    MSP216765   
9  Dec 14 2022  2:00PM  253122        10    MSP216766   

                             Customer ShipmentStatus  
0             GUSA Granules USA, Inc.       Released  
1      Sartorius Bioprocess Solutions       Released  
2  Sartorius Lab Products and Service       Released  
3                 AdvaGen Pharma, Ltd       Released  
4                    Methapharm, Inc.       Released  
5                    Methapharm, Inc.       Released  
6                    Methapharm, Inc.       Released  
7                    Methapharm, Inc.       Released  
8                    Methapharm, Inc.       Released  
9                    Methapharm, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
39  253122       Released         23
40  253123       Released          1
41  253124       Released          1
42  253125       Released          1
43  253126       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
253122                NaN        NaN      23.0
253123                NaN        NaN       1.0
253124                NaN        NaN       1.0
253125                NaN        NaN       1.0
253126                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
253035                0.0        0.0       1.0
253051                5.0        0.0      10.0
253053               14.0        1.0      10.0
253060                7.0       17.0       8.0
253062                9.0        1.0       2.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
253035                  0          0         1
253051                  5          0        10
253053                 14          1        10
253060                  7         17         8
253062                  9          1         2

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               253035          0          0         1
1               253051          5          0        10
2               253053         14          1        10
3               253060          7         17         8
4               253062          9          1         2

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               253035                            1
1               253051         5                 10
2               253053        14         1       10
3               253060         7        17        8
4               253062         9         1        2

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                            Customer
0   Dec 14 2022  2:28PM  253126        19             GUSA Granules USA, Inc.
1   Dec 14 2022  2:24PM  253125        16      Sartorius Bioprocess Solutions
2   Dec 14 2022  2:22PM  253124        16  Sartorius Lab Products and Service
3   Dec 14 2022  2:19PM  253123        19                 AdvaGen Pharma, Ltd
4   Dec 14 2022  2:00PM  253122        10                    Methapharm, Inc.
27  Dec 14 2022  1:51PM  253121        10                         Emerginnova
29  Dec 14 2022  1:47PM  253120        19                 AdvaGen Pharma, Ltd
33  Dec 14 2022  1:45PM  253119        10                TherapeuticsMD, Inc.
34  Dec 14 2022  1:33PM  253118        10            Senseonics, Incorporated
35  Dec 14 2022  1:31PM  253117        15         Virtus Pharmaceuticals, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                            Customer  \
0  Dec 14 2022  2:28PM  253126        19             GUSA Granules USA, Inc.   
1  Dec 14 2022  2:24PM  253125        16      Sartorius Bioprocess Solutions   
2  Dec 14 2022  2:22PM  253124        16  Sartorius Lab Products and Service   
3  Dec 14 2022  2:19PM  253123        19                 AdvaGen Pharma, Ltd   
4  Dec 14 2022  2:00PM  253122        10                    Methapharm, Inc.   
5  Dec 14 2022  1:51PM  253121        10                         Emerginnova   
6  Dec 14 2022  1:47PM  253120        19                 AdvaGen Pharma, Ltd   
7  Dec 14 2022  1:45PM  253119        10                TherapeuticsMD, Inc.   
8  Dec 14 2022  1:33PM  253118        10            Senseonics, Incorporated   
9  Dec 14 2022  1:31PM  253117        15         Virtus Pharmaceuticals, LLC   

  Completed Executing Released  
0                            1  
1                            1  
2                            1  
3                            1  
4                           23  
5                            2  
6                            4  
7                            1  
8                            1  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                            Customer  \
0  Dec 14 2022  2:28PM  253126        19             GUSA Granules USA, Inc.   
1  Dec 14 2022  2:24PM  253125        16      Sartorius Bioprocess Solutions   
2  Dec 14 2022  2:22PM  253124        16  Sartorius Lab Products and Service   
3  Dec 14 2022  2:19PM  253123        19                 AdvaGen Pharma, Ltd   
4  Dec 14 2022  2:00PM  253122        10                    Methapharm, Inc.   
5  Dec 14 2022  1:51PM  253121        10                         Emerginnova   
6  Dec 14 2022  1:47PM  253120        19                 AdvaGen Pharma, Ltd   
7  Dec 14 2022  1:45PM  253119        10                TherapeuticsMD, Inc.   
8  Dec 14 2022  1:33PM  253118        10            Senseonics, Incorporated   
9  Dec 14 2022  1:31PM  253117        15         Virtus Pharmaceuticals, LLC   

  Released Executing Completed  
0        1                      
1        1                      
2        1                      
3        1                      
4       23                      
5        2                      
6        4                      
7        1                      
8        1                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                            Customer  \
0  Dec 14 2022  2:28PM  253126        19             GUSA Granules USA, Inc.   
1  Dec 14 2022  2:24PM  253125        16      Sartorius Bioprocess Solutions   
2  Dec 14 2022  2:22PM  253124        16  Sartorius Lab Products and Service   
3  Dec 14 2022  2:19PM  253123        19                 AdvaGen Pharma, Ltd   
4  Dec 14 2022  2:00PM  253122        10                    Methapharm, Inc.   

  Released Executing Completed  
0        1                      
1        1                      
2        1                      
3        1                      
4       23

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                            Customer  \
0  Dec 14 2022  2:28PM  253126        19             GUSA Granules USA, Inc.   
1  Dec 14 2022  2:24PM  253125        16      Sartorius Bioprocess Solutions   
2  Dec 14 2022  2:22PM  253124        16  Sartorius Lab Products and Service   
3  Dec 14 2022  2:19PM  253123        19                 AdvaGen Pharma, Ltd   
4  Dec 14 2022  2:00PM  253122        10                    Methapharm, Inc.   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       1.0        NaN        NaN  
2       1.0        NaN        NaN  
3       1.0        NaN        NaN  
4      23.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                            Customer  \
0  Dec 14 2022  2:28PM  253126        19             GUSA Granules USA, Inc.   
1  Dec 14 2022  2:24PM  253125        16      Sartorius Bioprocess Solutions   
2  Dec 14 2022  2:22PM  253124        16  Sartorius Lab Products and Service   
3  Dec 14 2022  2:19PM  253123        19                 AdvaGen Pharma, Ltd   
4  Dec 14 2022  2:00PM  253122        10                    Methapharm, Inc.   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       1.0        0.0        0.0  
2       1.0        0.0        0.0  
3       1.0        0.0        0.0  
4      23.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         4049541     150.0       32.0        9.0
15         1518509      67.0        1.0       19.0
16          759284       3.0        0.0        0.0
19         2024799      18.0       17.0        7.0
20          506205       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  4049541     150.0       32.0        9.0
1        15  1518509      67.0        1.0       19.0
2        16   759284       3.0        0.0        0.0
3        19  2024799      18.0       17.0        7.0
4        20   506205       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     150.0       32.0        9.0
1        15      67.0        1.0       19.0
2        16       3.0        0.0        0.0
3        19      18.0       17.0        7.0
4        20       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  150.0
1        15  Released   67.0
2        16  Released    3.0
3        19  Released   18.0
4        20  Released    2.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    15   16    19   20
Status                                
Completed    9.0  19.0  0.0   7.0  0.0
Executing   32.0   1.0  0.0  17.0  0.0
Released   150.0  67.0  3.0  18.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    15   16    19   20
0          Completed    9.0  19.0  0.0   7.0  0.0
1          Executing   32.0   1.0  0.0  17.0  0.0
2           Released  150.0  67.0  3.0  18.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    15   16    19   20
0  Completed    9.0  19.0  0.0   7.0  0.0
1  Executing   32.0   1.0  0.0  17.0  0.0
2   Released  150.0  67.0  3.0  18.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()